In [ ]:
%sql
CREATE TABLE IF NOT EXISTS corebank_catalog.gold_corebank.gold_fraude_pix AS
WITH base AS (
  SELECT
    t.id_transacao,
    t.id_cliente,
    t.id_banco,
    t.id_banco_destino,
    t.id_cliente_destino,
    t.valor_transacao,
    t.dt_hr_transacao,
    t.dia_transacao,
    t.hora_transacao,
    coalesce(b_origem.instituicao, '') AS banco_origem,
    coalesce(b_dest.instituicao, '') AS banco_destino,
    COUNT(*) OVER (
      PARTITION BY t.id_cliente
      ORDER BY t.dt_hr_transacao
      RANGE BETWEEN INTERVAL 1 HOUR PRECEDING AND CURRENT ROW
    ) AS qtd_transacoes_1h,
    SUM(t.valor_transacao) OVER (
      PARTITION BY t.id_cliente
      ORDER BY t.dt_hr_transacao
      RANGE BETWEEN INTERVAL 1 HOUR PRECEDING AND CURRENT ROW
    ) AS soma_valor_1h
  FROM corebank_catalog.silver_corebank.silver_transacao_pix t
  LEFT JOIN corebank_catalog.silver_corebank.silver_clientes_anonimizados c
    ON t.id_cliente = c.id_cliente
  LEFT JOIN corebank_catalog.silver_corebank.silver_bancos b_origem
    ON t.id_banco = b_origem.id_banco
  LEFT JOIN corebank_catalog.silver_corebank.silver_bancos b_dest
    ON t.id_banco_destino = b_dest.id_banco
),
destinos_24h AS (
  SELECT
    t1.id_transacao,
    COUNT(DISTINCT t2.id_cliente_destino) AS qtd_destinos_24h
  FROM corebank_catalog.silver_corebank.silver_transacao_pix t1
  LEFT JOIN corebank_catalog.silver_corebank.silver_transacao_pix t2
    ON t1.id_cliente = t2.id_cliente
    AND t2.dt_hr_transacao BETWEEN t1.dt_hr_transacao - INTERVAL 24 HOURS AND t1.dt_hr_transacao
  GROUP BY t1.id_transacao
)
SELECT
  base.*,
  coalesce(destinos_24h.qtd_destinos_24h, 0) AS qtd_destinos_24h,
  CASE WHEN valor_transacao > 5000 THEN 1 ELSE 0 END AS flag_valor_alto,
  CASE WHEN hora_transacao < 6 OR hora_transacao > 22 THEN 1 ELSE 0 END AS flag_horario_suspeito,
  CASE WHEN qtd_transacoes_1h >= 10 THEN 1 ELSE 0 END AS flag_rajada_transacoes,
  CASE WHEN coalesce(destinos_24h.qtd_destinos_24h, 0) >= 5 THEN 1 ELSE 0 END AS flag_muitos_destinos,
  CASE
    WHEN (valor_transacao > 5000 AND (hora_transacao < 6 OR hora_transacao > 22))
      OR qtd_transacoes_1h >= 10
      OR coalesce(destinos_24h.qtd_destinos_24h, 0) >= 5
    THEN 'ALTO_RISCO'
    ELSE 'BAIXO_RISCO'
  END AS risco_fraude
FROM base
LEFT JOIN destinos_24h
  ON base.id_transacao = destinos_24h.id_transacao